In [600]:
import pandas as pd
import numpy as np

In [601]:
becarios = pd.read_csv("test_data.csv")

In [602]:
becarios['NOMBRE'] = becarios['NOMBRE'].astype(str)
becarios['Editar'] = [[] for _ in range(len(becarios.index))]

In [603]:
percent_essays_buddy = 0.0
percent_essays_group = 0.0
percent_essays_everyone = 100 - percent_essays_buddy - percent_essays_group

percent_essays_buddy /= 100
percent_essays_group /= 100
percent_essays_everyone /= 100

def numero_ensayos(ensayos, porcentaje):
    return int(np.ceil(ensayos*porcentaje))

def ensayo_asignado(escritor, editor):
    becarios.at[escritor, 'Ensayos Personales'] = becarios.at[escritor, 'Ensayos Personales'] -1
    becarios.at[editor, 'Ensayos a Editar'] = becarios.at[editor, 'Ensayos a Editar'] -1
    
def get_ensayos_a_editar(editor):
    return becarios.at[becarios.index.get_loc(editor.name),'Ensayos a Editar']
    

In [604]:
for index, row in becarios.iterrows():
    buddy = becarios[(becarios['Area']==row['Area']) & (becarios['NOMBRE'] != row['NOMBRE'])].sample(n=1).iloc[0]
    row['Editar'].append(buddy['NOMBRE'])
    ensayo_asignado(becarios.index.get_loc(row.name), becarios.index.get_loc(buddy.name))
    ensayos_buddy = numero_ensayos(row['Ensayos Personales'],percent_essays_buddy)
    for i in range(ensayos_buddy):
        if get_ensayos_a_editar(buddy) > 0:
            row['Editar'].append(buddy['NOMBRE'])
            ensayo_asignado(becarios.index.get_loc(row.name), becarios.index.get_loc(buddy.name))

In [605]:
for index, row in becarios.iterrows():
    ensayos_misma_area = numero_ensayos(row['Ensayos Personales'],percent_essays_group)
    misma_area = becarios[(becarios['Area']==row['Area']) & (becarios['NOMBRE'] != row['Editar'][-1]) &(becarios['NOMBRE'] != row['NOMBRE'])].sample(n=ensayos_misma_area,replace=True)
    for i in range(ensayos_misma_area):
        if get_ensayos_a_editar(misma_area.iloc[i]) > 0:
            row['Editar'].append(misma_area.iloc[i]['NOMBRE'])
            ensayo_asignado(becarios.index.get_loc(row.name), becarios.index.get_loc(misma_area.iloc[i].name))

In [606]:
for index, row in becarios.iterrows():
    ensayos_otra_area = numero_ensayos(row['Ensayos Personales'],percent_essays_everyone)
    otra_area = becarios[(becarios['Area']!=row['Area'])].sample(n=ensayos_otra_area,replace=True)
    for i in range(int(ensayos_otra_area)):
        if get_ensayos_a_editar(otra_area.iloc[i]) > 0:
            row['Editar'].append(otra_area.iloc[i]['NOMBRE'])
            ensayo_asignado(becarios.index.get_loc(row.name), becarios.index.get_loc(otra_area.iloc[i].name))